<a href="https://colab.research.google.com/github/crmccc/bilibili_heartRank_spider/blob/main/50M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
BEGIN_NUMBER = 70000000 #@param {type:"slider", min:0, max:200000000, step:10000000}
END_NUMBER = 60000000 #@param {type:"slider", min:0, max:200000000, step:10000000}
# -*- coding: utf-8 -*-
'''
Little spaider crawling bilibili live room medalRank
automatic save prograss,support contiune from break point
usie api: http://api.live.bilibili.com/rankdb/v1/RoomRank/webMedalRank?ruid=2
can do around 1,000~1,200 per min

file tree:
----..
    \heartrankData
        HEAD               <-storage prograss
        heartrank.txt      <-storage data
    heartrank_distro.py    <-script

@author:cyan
@license: WTFL
'''
#connect to self drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive')

import asyncio
import random
import threading
import time
import queue
import requests

#BEGIN_NUMBER = 50000000
#END_NUMBER =  200000000
MIN_LEVEL = 25  # !minium level to write
DATA_FILE = f'heartrank{BEGIN_NUMBER//10000000}M-{END_NUMBER//10000000}.txt'
COMPLETE_FILE = 'heartrank_complete.txt'
# DATA_FILE = f'heartrank{BEGIN_NUMBER/1000000}M-{END_NUMBER/1000000}M.txt'
# COMPLETE_FILE=f'heartrank{BEGIN_NUMBER/1000000}M-{END_NUMBER/1000000}M_complete.txt'
HEAD_FILE = 'HEAD'  # !store prograss
#TARGET_FILE = 'TARGET'
PATH = f'heartrank{BEGIN_NUMBER//10000000}M-{END_NUMBER//10000000}/'  # !
STEP = 200  # test says it's best,donot go over 500
STEP_COUNT_PER_THREAD = 5
THREAD_NUMBER = 32  # how many thread it use
URL = 'http://api.live.bilibili.com/rankdb/v1/RoomRank/webMedalRank'

heads = [0 for i in range(0, THREAD_NUMBER)]
targets = [0 for i in range(0, THREAD_NUMBER)]

USER_AGENT_LIST = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:22.0) Gecko/20130328 Firefox/22.0"
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1464.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36"
    "Mozilla/5.0 (X11; NetBSD) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36"
    "Mozilla/5.0 (Windows NT 5.0; rv:21.0) Gecko/20100101 Firefox/21.0"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36"
    "Opera/9.80 (Windows NT 5.1; U; cs) Presto/2.7.62 Version/11.01"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1944.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20130331 Firefox/21.0"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36"
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36"
    "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.93 Safari/537.36"
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36"
    "Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.137 Safari/4E423F"
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36"
]


def get_headers():
    # randomly draw a header from list

    return {"user-agent": random.choice(USER_AGENT_LIST)}


def get_last_line(inputfile: str):
    #!deprecated
    filesize = os.path.getsize(inputfile)
    blocksize = 1024
    dat_file = open(inputfile, 'rb')
    last_line = ""
    if filesize > blocksize:
        maxseekpoint = (filesize // blocksize)
        dat_file.seek((maxseekpoint - 1) * blocksize)
    elif filesize:
        # maxseekpoint = blocksize % filesize
        dat_file.seek(0, 0)
    lines = dat_file.readlines()
    if lines:
        last_line = lines[-1].strip()
    # print "last line : ", last_line
    dat_file.close()
    return last_line


def udpate_head(count, number, q):
    heads[count] = number
    tmp=(count,number)
    q.put(tmp)


def udpate_head_safe(count, number):
    heads[count] = number
    with open(PATH+HEAD_FILE+'-'+str(count), 'w') as wahtnow:  # wirte prograss to HEAD
        wahtnow.write(str(number))


def get_head_from_file(count):
    with open(PATH+HEAD_FILE+'-'+str(count), 'r') as wahtnow:
        heads[count] = int(wahtnow.read())
    return heads[count]


def update_target(count):
    targets[count] = heads[count]+(STEP*STEP_COUNT_PER_THREAD)
    return targets[count]
    # with open(PATH+TARGET_FILE+'-'+str(count), 'w') as wahtnow:  # wirte prograss to HEAD
    #    wahtnow.write(str(number))
    #    wahtnow.close()


def get_chunk(count):
    begin = get_head_from_file(count)
    heads[count] = begin
    targets[count] = begin+STEP*STEP_COUNT_PER_THREAD

    return heads[count], targets[count]


def get_next_chunk(begin):
    begin += STEP*THREAD_NUMBER*STEP_COUNT_PER_THREAD
    return begin,begin+STEP_COUNT_PER_THREAD*STEP


async def get_medal(ruid, s, threat_count, result_q):
    # get data from bilibil and proccess

    respond = s.get(url=URL, params={'ruid': str(ruid)})
    retry_count = 1

    while respond.status_code != 200 and retry_count < 5:
        respond = s.get(url=URL, params={'ruid': str(ruid)})
        retry_count = retry_count+1
        print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] E : Request for {ruid} FAILD,retry count: {retry_count}\n', end='')
        time.sleep(0.5)

    if respond.json()['data']['list']:
        top_data = respond.json()['data']['list'][0]
        if top_data['level'] >= MIN_LEVEL:
            normaled_str = f"{str(ruid)} {str(top_data['uid'])} {str(top_data['uname'])} {str(top_data['medal_name'])} {str(top_data['level'])}"
            print(
                f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : thread[{threat_count}] found : {normaled_str}.\n', end='')
            result_q.put(normaled_str)


def initilze(q):

    for count in range(0, THREAD_NUMBER):
        # create HEAD on frist run
        if not os.path.exists(PATH+HEAD_FILE+'-'+str(count)):
            print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : file :{PATH+HEAD_FILE}-{str(count)} not found,create.\n', end='')
            udpate_head_safe(count, BEGIN_NUMBER+STEP *
                             STEP_COUNT_PER_THREAD*(count))
            update_target(count)


def main_mt(threat_count: int, q):
    begin_number, end_number = get_chunk(threat_count)
    s = requests.Session()
    s.headers = get_headers()  # change header every thread

    # if ((get_last_line(DATA_FILE))):
    #     begin_number = int(
    #         str(((get_last_line(DATA_FILE))), 'utf-8').split(' ')[0])+1
    while end_number < END_NUMBER:
        i = begin_number
        round_counter = 1
        while i < end_number:
            print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : thread[{threat_count}] going : {i} (chunk:{round_counter}/{STEP_COUNT_PER_THREAD}).\n', end='')
            loop = asyncio.new_event_loop()
            tasks = [get_medal(ruid, s, threat_count, q)
                     for ruid in range(i, i+STEP)]
            loop.run_until_complete(asyncio.wait(tasks))
            loop.stop()
            i += STEP
            round_counter += 1
        begin_number, end_number = get_next_chunk(begin_number)
        udpate_head(threat_count,begin_number,q)
        # os.rename(PATH+str(threat_count)+'-'+DATA_FILE,PATH+str(threat_count)+'-'+COMPLETE_FILE) #rename the result file once complete
        # os.remove(PATH+HEAD_FILE+'-'+str(threat_count))
        print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : thread[{threat_count}] going : {begin_number} to {i} done.\n', end='')


class worker_thread (threading.Thread):
    def __init__(self, count, result_q):
        threading.Thread.__init__(self)
        self.counter = count
        self.result_q=result_q
        # self.begin_number = begin_number
        # self.end_number = steps*STEP+begin_number

    def run(self) -> None:
        # main_mt(self.counter, self.begin_number, self.end_number)

        main_mt(self.counter, self.result_q)


class file_joiner_thread(threading.Thread):
    def __init__(self, result_q) -> None:
        threading.Thread.__init__(self)
        self.fo = open(f'{PATH}{DATA_FILE}', 'a')
        self.q = result_q

    def run(self):
        counter = 1
        while True:
            res = self.q.get()
            if type(res) is tuple:
                # wirte prograss to HEAD
                with open(PATH+HEAD_FILE+'-'+str(res[0]), 'w') as wahtnow:
                    wahtnow.write(str(res[1]))
                print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : file joiner: update : {HEAD_FILE+"-"+str(res[0])}.\n', end='')
            elif type(res) is str :
                print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : file joiner: going :  join files.\n', end='')
                self.fo.write(res+'\n')
                self.fo.flush()


def main():
    if PATH and (not os.path.exists(PATH)):
        os.mkdir(PATH)
        print(f'[{time.strftime("%m.%d %H:%M:%S", time.localtime())}] I : path :{PATH} not found,create.\n', end='')

    result_q = queue.Queue()

    joiner = file_joiner_thread( result_q)
    joiner.start()

    initilze(result_q)
    threads = []
    for count in range(0, THREAD_NUMBER):
        t = worker_thread(count,result_q)
        t.start()
        threads.append(t)
    while True:
        for count in range(0, THREAD_NUMBER):
            if not t.is_alive():
                return


main()
# snyclize()
